##Ingest

In [ ]:
import numpy as np
import pandas as pd
import string

In [ ]:
df=pd.read_json('News_Category_Dataset_v2.json',lines=True)

##Preprocessing

In [ ]:
lis=[ 'WORLD NEWS',
       'SPORTS', 'MONEY',
       'WOMEN','SCIENCE']

In [ ]:
df=df.loc[df['category'].isin(lis)]
df=df.reset_index(drop=True)

In [ ]:
df["text"] = df["headline"] + df["short_description"] 

In [ ]:

df['text'][1]


In [ ]:
def removePunct(txt):
  txt_nopunct="".join([c for c in txt if c not in string.punctuation])
  return txt_nopunct

In [ ]:
df['textClean']=df['text'].apply(lambda x:removePunct(x))
df.head()

In [ ]:
import re

In [ ]:
def tokenize(txt):
  tokens=re.split('\W+',txt)
  return tokens

In [ ]:
df['textCleanToken']=df['textClean'].apply(lambda x:tokenize(x.lower()))
df.head()

In [ ]:
import nltk
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')

In [ ]:
def removeStopwords(txt):
  txt_clean=[word for word in txt if word not in stopwords]
  return txt_clean

In [ ]:
df['noStopwords']=df['textCleanToken'].apply(lambda x: removeStopwords(x))
df.head()

In [ ]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()
def stemming(token_txt):
  text=[ps.stem(word) for word in token_txt]
  return text

In [ ]:
df['stemmed']=df['noStopwords'].apply(lambda x: stemming(x))
df.head()

In [ ]:
def cleanWords(text):
  txt="".join([p for p in text if p not in string.punctuation])
  tokens=re.split("\W+",txt)
  txt=[ps.stem(word) for word in tokens if word not in stopwords]

In [ ]:
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [ ]:
df['string']=df['stemmed'].apply(lambda x: listToString(x))
df.head()

In [ ]:
def removeNum(txt):
  result = ''.join([i for i in txt if not i.isdigit()])
  return result

In [ ]:
df['Nonum']=df['string'].apply(lambda x: removeNum(x))

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df

##Doc2Vec

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
import nltk


nltk.download('punkt')
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(df['Nonum'])]

In [ ]:
tagged_data

In [ ]:
max_epochs = 100
vec_size = 200
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

##CNN

In [ ]:
nnvec=model.docvecs.vectors_docs

In [ ]:
nnvecdf = pd.DataFrame(nnvec)

In [ ]:
y=df['category']

In [ ]:

x=nnvecdf.values

In [ ]:
import tensorflow as tf


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

yLE = le.fit_transform(y)

In [ ]:
yLE = np.asarray( tf.keras.utils.to_categorical(yLE))

In [ ]:
yLE

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainy, testy = train_test_split(x, yLE, test_size=0.25, random_state=1000)

In [ ]:
trainX

array([[-0.09309758,  0.28601527, -0.84262973, ..., -0.42032093,
        -0.5559607 ,  0.05260021],
       [-0.37408864, -1.3017676 , -0.21571033, ..., -1.3674392 ,
         0.67712283,  1.3236877 ],
       [-0.16388178,  0.56549436, -1.6666541 , ..., -0.20130236,
        -0.50973576,  0.94197243],
       ...,
       [-0.44317573,  0.27023143, -1.8648895 , ..., -0.48540053,
         0.5698723 ,  1.7259253 ],
       [-0.04157736,  3.5654485 , -0.3156717 , ..., -0.97866446,
        -1.3631786 , -1.1497347 ],
       [-0.39830184,  0.61163926, -0.48205402, ..., -0.27936703,
        -0.11220853,  0.47539946]], dtype=float32)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
import pickle

In [ ]:
trainX.shape

(10827, 200)

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
trainX

array([[-0.09309758,  0.28601527, -0.84262973, ..., -0.42032093,
        -0.5559607 ,  0.05260021],
       [-0.37408864, -1.3017676 , -0.21571033, ..., -1.3674392 ,
         0.67712283,  1.3236877 ],
       [-0.16388178,  0.56549436, -1.6666541 , ..., -0.20130236,
        -0.50973576,  0.94197243],
       ...,
       [-0.44317573,  0.27023143, -1.8648895 , ..., -0.48540053,
         0.5698723 ,  1.7259253 ],
       [-0.04157736,  3.5654485 , -0.3156717 , ..., -0.97866446,
        -1.3631786 , -1.1497347 ],
       [-0.39830184,  0.61163926, -0.48205402, ..., -0.27936703,
        -0.11220853,  0.47539946]], dtype=float32)

In [ ]:
import numpy as np
np.amin(trainX)

-5.2274995

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# transform data
trainX = scaler.fit_transform(trainX)

In [ ]:
testX = scaler.fit_transform(testX)

In [100]:
max_features =200
embedding_dim =200
sequence_length = 201

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features +1, embedding_dim, 
                                    embeddings_regularizer = regularizers.l2(0.0005)))                                    

model.add(tf.keras.layers.Conv1D(360,3, activation='relu',\
                                 kernel_regularizer = regularizers.l2(0.0005),\
                                 bias_regularizer = regularizers.l2(0.0005)))                               


model.add(tf.keras.layers.GlobalMaxPooling1D())

model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(5, activation='sigmoid',\
                                kernel_regularizer=regularizers.l2(0.001),\
                                bias_regularizer=regularizers.l2(0.001),))
                               



model.summary()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer='Nadam', metrics=["CategoricalAccuracy"])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 200)         40200     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 360)         216360    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 360)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 360)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1805      
Total params: 258,365
Trainable params: 258,365
Non-trainable params: 0
_________________________________________________________________


In [102]:
epochs = 20
# Fit the model using the train and test datasets.
history = model.fit(trainX, trainy,validation_data= (testX,testy),epochs=epochs )

Epoch 1/20
339/339 [==============================] - 44s 131ms/step - loss: 1.5342 - categorical_accuracy: 0.3403 - val_loss: 1.5381 - val_categorical_accuracy: 0.3325
Epoch 2/20
339/339 [==============================] - 44s 131ms/step - loss: 1.5343 - categorical_accuracy: 0.3403 - val_loss: 1.5375 - val_categorical_accuracy: 0.3325
Epoch 3/20
339/339 [==============================] - 44s 131ms/step - loss: 1.5344 - categorical_accuracy: 0.3403 - val_loss: 1.5368 - val_categorical_accuracy: 0.3325
Epoch 4/20
339/339 [==============================] - 44s 131ms/step - loss: 1.5339 - categorical_accuracy: 0.3403 - val_loss: 1.5368 - val_categorical_accuracy: 0.3325
Epoch 5/20
339/339 [==============================] - 44s 130ms/step - loss: 1.5342 - categorical_accuracy: 0.3403 - val_loss: 1.5372 - val_categorical_accuracy: 0.3325
Epoch 6/20
339/339 [==============================] - 44s 131ms/step - loss: 1.5344 - categorical_accuracy: 0.3403 - val_loss: 1.5370 - val_categorical_acc